In [1]:
%matplotlib qt
import scipy
from scipy.io import wavfile
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal
import osascript
from gtts import gTTS 
import os 
import pyaudio
import wave
import keyboard  # using module keyboard
import soundfile as sf
import math
import pyloudnorm as pyln
from sys import byteorder
from array import array
from struct import pack
import librosa
from scipy.signal import butter, sosfiltfilt
import pysptk
import python_speech_features
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 
import time
import sherpa
import sherpa.algorithms.bayesian_optimization as bayesian_optimization
from sklearn.model_selection import KFold
from keras.optimizers import Adadelta
from keras.layers import Conv2D
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
import keras as keras
from  conch.analysis.formants import lpc
import sklearn
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers
from keras import backend as K

Using TensorFlow backend.


In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = np.load("../../splitdata.npy", allow_pickle=True)

In [ ]:
dataset, fs, y = np.load("../../CREMA/CREMAdatafsy.npy", allow_pickle=True)

In [ ]:
chunk = int(512*(fs/1000))
one_point = dataset[0][0:chunk]
one_point.shape, fs, chunk

In [ ]:
def remove_silence_from(amplitudes, threshold):
    silenced = []
    for x in amplitudes:
        if x >= threshold:
            silenced.append(x)
    return silenced

In [ ]:
def MFCC_algorithm(np_data, fs):
    MFCCs = []
    print("running .....")
    #for progess bar
    for one_sound in np_data:
        
        one_sound = np.asarray(one_sound).reshape(int((fs/1000)*512), 1)
        
        MFCCs.append(python_speech_features.base.mfcc(one_sound, samplerate=fs, 
                                     winlen=0.025, winstep=0.01, numcep=13, 
                                     nfilt=26, nfft=552))
    return np.array(MFCCs)

In [ ]:
def get_pitch_vector(data, fs):
    data = np.float32(data)
    pitch = pysptk.sptk.rapt(y=data, sr=fs, hopsize = 40)
    silenced = remove_silence_from(pitch, np.mean(pitch))
    return silenced

In [ ]:
def get_spectral_vector(data, fs):
    data = np.float32(data)
    cent = librosa.feature.spectral_centroid(y=data, sr=fs, hop_length=165)
    return cent

In [ ]:
def get_rms_vector(data):
    temp_data = np.float32(data)
    cent = librosa.feature.rms(y=temp_data, hop_length=165)
    return cent

In [ ]:
def get_zero_vector(data):
    temp_data = np.float32(data)
    cent = librosa.feature.zero_crossing_rate(y=temp_data, hop_length=165)
    return cent

In [ ]:
def get_sr_vector(data):
    temp_data = np.float32(data)
    cent = librosa.feature.spectral_rolloff(y=temp_data, hop_length=165)
    return cent

In [ ]:
def me_and_gradient(x):
    return x, np.gradient(x)

In [ ]:
print(len((get_spectral_vector(one_point, fs))[0]))

print(len(get_spectral_vector(one_point, fs)[0]))

print(len(get_zero_vector(one_point)[0]))

print(len(get_rms_vector(one_point)[0]))

print(len(get_sr_vector(one_point)[0]))


In [3]:
np.save("Y_BOG_SPEC", y)

In [ ]:
fs = 16000
dataset, y, useless_number = np.load("../../CREMA_chunked.npy", allow_pickle=True)

KeyboardInterrupt: 

E0801 16:56:42.455740 140735549535104 alias.py:221] Invalid alias: The name clear can't be aliased because it is another magic command.
E0801 16:56:42.458575 140735549535104 alias.py:221] Invalid alias: The name more can't be aliased because it is another magic command.
E0801 16:56:42.459796 140735549535104 alias.py:221] Invalid alias: The name less can't be aliased because it is another magic command.
E0801 16:56:42.460480 140735549535104 alias.py:221] Invalid alias: The name man can't be aliased because it is another magic command.


In [ ]:
X = MFCC_algorithm(dataset, fs)

In [ ]:
np.save("X_CREMA_MFCC", X)

In [ ]:
X = X.reshape(X.shape[0], 50, 13, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)

In [ ]:
np.save("X_Y_SPLIT.npy", (X_train,X_val, X_test, y_train,y_val, y_test))

In [2]:
(X_train,X_val, X_test, y_train,y_val, y_test) = np.load("X_Y_SPLIT.npy", allow_pickle=True)

In [3]:
print("x train shape: " +str(X_train.shape))
print("y train shape: " +str(y_train.shape))
print("x test shape: " +str(X_test.shape))
print("y test shape: " +str(y_test.shape))

x train shape: (32499, 50, 13, 1)
y train shape: (32499, 12)
x test shape: (4063, 50, 13, 1)
y test shape: (4063, 12)


In [4]:
def cnn_MFCC_LSTM(X_train, X_test, y_train, y_test, regularization = 0.1,drop_likely = 0.1,
              learning_rate = 0.001, beta1 = 0.9, beta2 = 0.999,
             num_iterations = 40, size_batch=64,activation = 'relu'):
    # Set the target class number
    
    # VGG 19
    
    target_class = y_train.shape[1]
    # Model 
    model = Sequential()
    
    model.add(keras.layers.BatchNormalization())    
    
    model.add(Conv2D(64, kernel_size = (3,3), activation=activation, input_shape = (50,13,1)))
    model.add(Conv2D(64, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
    model.add(keras.layers.BatchNormalization())
    
    model.add(Conv2D(128, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(128, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
    model.add(keras.layers.BatchNormalization())
    
    model.add(Conv2D(256, kernel_size = (3,3), activation=activation))
    model.add(Conv2D(256, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(256, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(256, kernel_size = (3,3), activation=activation))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
    model.add(keras.layers.BatchNormalization())
    
    
    model.add(Conv2D(512, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(512, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(512, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(512, kernel_size = (3,3), activation=activation))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    model.add(keras.layers.BatchNormalization())
    
    
    
    model.add(Flatten())

    
    
    model.add(Dense(4096, activation=activation))
    model.add(Dense(4096, activation=activation))
    model.add(Dense(256, activation=activation))
    
    
    
    model.add(Dense(target_class, activation='softmax'))

    
    
    es = keras.callbacks.EarlyStopping(monitor='val_acc', mode='max',  patience=10, min_delta=0)
    
    opt = keras.optimizers.Adam(lr=learning_rate, beta_1=beta1, beta_2=beta2, epsilon=10e-8)
       
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt,metrics=[keras.metrics.categorical_accuracy])
    
    nn_history = model.fit(X_train, y_train, batch_size=size_batch, epochs=num_iterations, 
                         validation_data=(X_test, y_test),
                          callbacks=[es])
    
    a = model.predict(X_train)
    predictions = np.zeros_like(a)
    predictions[np.arange(len(a)), a.argmax(1)] = 1
    print(predictions)
    print(y_train)
    print(classification_report(y_train,predictions))
    a = model.predict(X_test)
    predictions = np.zeros_like(a)
    predictions[np.arange(len(a)), a.argmax(1)] = 1
    print(classification_report(y_test,predictions))
    
    
    

    return model

W0802 11:17:18.590553 140735549535104 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0802 11:17:18.767642 140735549535104 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0802 11:17:18.787890 140735549535104 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0802 11:17:18.988568 140735549535104 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



[]

In [6]:
def simple_cnn(X_train, X_test, y_train, y_test, regularization = 0.1,drop_likely = 0.1,
              learning_rate = 0.001, beta1 = 0.9, beta2 = 0.999,
             num_iterations = 40, size_batch=128,activation = 'tanh'):
    target_class = y_train.shape[1]
    # Model 
    model = Sequential()
    
    model.add(keras.layers.BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), activation=activation, input_shape = (50,13,1)))
    
    model.add(Conv2D(128, kernel_size = (3,3), activation=activation))

    model.add(Conv2D(128, kernel_size = (3,3), activation=activation))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
    
    
    model.add(Flatten())
    
    model.add(Dense(2048, activation=activation))
    model.add(Dense(1024, activation=activation))
    model.add(Dense(512, activation=activation))
    
    model.add(Dense(target_class, activation='softmax'))

    
    es = keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', mode='max',  patience=10, min_delta=0)
    
    opt = keras.optimizers.Adam(lr=learning_rate, beta_1=beta1, beta_2=beta2, epsilon=10e-8)
       
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt,metrics=[keras.metrics.categorical_accuracy])
    
    nn_history = model.fit(X_train, y_train, batch_size=size_batch, epochs=num_iterations, 
                         validation_data=(X_test, y_test),
                          callbacks=[es])
    
    a = model.predict(X_train)
    predictions = np.zeros_like(a)
    predictions[np.arange(len(a)), a.argmax(1)] = 1
    print(predictions)
    print(y_train)
    print(classification_report(y_train,predictions))
    a = model.predict(X_test)
    predictions = np.zeros_like(a)
    predictions[np.arange(len(a)), a.argmax(1)] = 1
    print(classification_report(y_test,predictions))
    
    
    

    return model

In [ ]:
simple_cnn(X_train, X_test, y_train, y_test)

Train on 32499 samples, validate on 4063 samples
Epoch 1/40
32499/32499 [==============================] - 721s 22ms/step - loss: 2.5986 - categorical_accuracy: 0.0875 - val_loss: 2.5069 - val_categorical_accuracy: 0.0925
Epoch 2/40
32499/32499 [==============================] - 981s 30ms/step - loss: 2.5065 - categorical_accuracy: 0.0893 - val_loss: 2.5363 - val_categorical_accuracy: 0.0817
Epoch 3/40
24960/32499 [======================>.......] - ETA: 3:37 - loss: 2.5147 - categorical_accuracy: 0.0875

E0729 11:00:10.212630 140735522395008 ultratb.py:149] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-07b4934d80f8>", line 1, in <module>
    simple_cnn(X_train, X_test, y_train, y_test)
  File "<ipython-input-6-d18395cd049d>", line 40, in simple_cnn
    callbacks=[es])
  File "/usr/local/lib/python3.7/site-packages/keras/engine/training.py", line 1039, in fit
    validation_steps=validation_steps)
  File "/usr/local/lib/python3.7/site-packages/keras/engine/training_arrays.py", line 199, in fit_loop
    outs = f(ins_batch)
  File "/usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py", line 2715, in __call__
    return self._call(inputs)
  File "/usr/local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py", line 2675, in _call
    fetched = self._callable_fn(*array_vals)
  File "/usr/local/lib/python3.7/site-packages/tensorflow/python/c

KeyboardInterrupt: 

E0729 11:00:10.812683 140735522395008 alias.py:221] Invalid alias: The name clear can't be aliased because it is another magic command.
E0729 11:00:10.813905 140735522395008 alias.py:221] Invalid alias: The name more can't be aliased because it is another magic command.
E0729 11:00:10.814826 140735522395008 alias.py:221] Invalid alias: The name less can't be aliased because it is another magic command.
E0729 11:00:10.815836 140735522395008 alias.py:221] Invalid alias: The name man can't be aliased because it is another magic command.


In [ ]:
cnn_MFCC_LSTM(X_train, X_val, y_train, y_val)

In [11]:
np.reshape(X_train, (X_train.shape[0], 50*13))

array([[ 1.04027690e+01, -2.29216742e+00, -2.45621438e+01, ...,
         4.88963861e+00,  1.48930418e+01,  9.20361442e+00],
       [ 1.13726075e+01,  1.64783184e-01, -2.58792595e+01, ...,
        -2.06376344e+00,  1.46754733e+01,  7.15964668e+00],
       [ 1.36984766e+01, -3.23827023e+00, -2.46365405e+01, ...,
        -9.64290945e-01,  4.47336320e+00,  8.45182667e+00],
       ...,
       [ 1.89949429e+01,  2.19963632e+01, -2.23273079e+01, ...,
        -6.88303927e+00,  2.70539077e+01,  4.76705165e+00],
       [ 1.49043370e+01,  9.61019133e+00, -3.04150089e+01, ...,
        -2.10617805e-02, -5.44090491e+00, -1.51162092e+00],
       [ 1.03970799e+01, -2.06840049e+00, -1.48779458e+01, ...,
         4.57743321e+00,  2.12141676e+01,  1.02466831e+01]])

In [40]:
def simple_1dconv(X_train, X_test, y_train, y_test, regularization = 0.1,drop_likely = 0.1,
              learning_rate = 0.001, beta1 = 0.9, beta2 = 0.999,
             num_iterations = 40, size_batch=128,activation = 'relu'):
    target_class = y_train.shape[1]
    X_train = np.reshape(X_train, (X_train.shape[0], 50*13,1))
    X_test =np.reshape(X_test, (X_test.shape[0], 50*13,1))
    
    model = Sequential()
    model.add(Conv1D(256, 5, padding='same',input_shape=(X_train.shape[1],1))) #1
    model.add(Activation(activation))
    model.add(Conv1D(256, 5, padding='same')) #2
    model.add(keras.layers.BatchNormalization())    
    model.add(Activation(activation))
    model.add(Dropout(0.25))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(128, 5, padding='same')) #3
    model.add(Activation(activation)) 
    model.add(Conv1D(128, 5, padding='same')) #4
    model.add(Activation(activation))
    model.add(Conv1D(128, 5, padding='same')) #5
    model.add(Activation(activation))
    model.add(Conv1D(128, 5, padding='same')) #6
    model.add(keras.layers.BatchNormalization())    
    model.add(Activation(activation))
    model.add(Dropout(0.25))
    model.add(MaxPooling1D(pool_size=(8)))
    model.add(Conv1D(64, 5, padding='same')) #7
    model.add(Activation(activation))
    model.add(Conv1D(64, 5, padding='same')) #8
    model.add(Activation(activation))

    
    model.add(Flatten())
    
    
    model.add(Dense(target_class)) #9
    model.add(Activation('softmax'))
    
    
    es = keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', mode='max',  patience=10, min_delta=0)
    
    opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
    
#     opt = keras.optimizers.Adam(lr=learning_rate, beta_1=beta1, beta_2=beta2, epsilon=10e-8)
       
    lr_reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.9, patience=20, min_lr=0.000001)
    model.compile(loss='categorical_crossentropy',optimizer=opt)
    cnnhistory=model.fit(X_train, y_train, batch_size=16, epochs=700, validation_data=(X_test, y_test), 
                             callbacks=[lr_reduce])

    
    a = model.predict(X_train)
    predictions = np.zeros_like(a)
    predictions[np.arange(len(a)), a.argmax(1)] = 1
    print(predictions)
    print(y_train)
    print(classification_report(y_train,predictions))
    a = model.predict(X_test)
    predictions = np.zeros_like(a)
    predictions[np.arange(len(a)), a.argmax(1)] = 1
    print(classification_report(y_test,predictions))
    
    
    
    return model

In [ ]:
simple_1dconv(X_train, X_test, y_train, y_test)

Train on 32499 samples, validate on 4063 samples
Epoch 1/700
32499/32499 [==============================] - 946s 29ms/step - loss: 2.4967 - val_loss: 2.4478
Epoch 2/700
32499/32499 [==============================] - 646s 20ms/step - loss: 2.3753 - val_loss: 2.4057
Epoch 3/700
32499/32499 [==============================] - 647s 20ms/step - loss: 2.2933 - val_loss: 2.3426
Epoch 4/700
32499/32499 [==============================] - 649s 20ms/step - loss: 2.2295 - val_loss: 2.3211
Epoch 5/700
32499/32499 [==============================] - 674s 21ms/step - loss: 2.1756 - val_loss: 2.2523
Epoch 6/700
32499/32499 [==============================] - 651s 20ms/step - loss: 2.1436 - val_loss: 2.2837
Epoch 7/700
32499/32499 [==============================] - 655s 20ms/step - loss: 2.1164 - val_loss: 2.1921
Epoch 8/700
32499/32499 [==============================] - 615s 19ms/step - loss: 2.0928 - val_loss: 2.2812
Epoch 9/700
32499/32499 [==============================] - 582s 18ms/step - loss: 2.083

Epoch 76/700
32499/32499 [==============================] - 585s 18ms/step - loss: 1.8409 - val_loss: 2.1548
Epoch 77/700
32499/32499 [==============================] - 583s 18ms/step - loss: 1.8383 - val_loss: 1.9976
Epoch 78/700
32499/32499 [==============================] - 581s 18ms/step - loss: 1.8368 - val_loss: 2.0857
Epoch 79/700
32499/32499 [==============================] - 578s 18ms/step - loss: 1.8394 - val_loss: 2.1315
Epoch 80/700
32499/32499 [==============================] - 579s 18ms/step - loss: 1.8346 - val_loss: 1.9794
Epoch 81/700
32499/32499 [==============================] - 581s 18ms/step - loss: 1.8347 - val_loss: 2.1766
Epoch 82/700
32499/32499 [==============================] - 583s 18ms/step - loss: 1.8325 - val_loss: 1.9936
Epoch 83/700
32499/32499 [==============================] - 589s 18ms/step - loss: 1.8320 - val_loss: 2.1384
Epoch 84/700
32499/32499 [==============================] - 582s 18ms/step - loss: 1.8325 - val_loss: 2.1415
Epoch 85/700
32499/

In [ ]:
parameters = [sherpa.Continuous(name='lr', range=[0.0001, 0.1], scale='log'),
              sherpa.Continuous(name='beta1', range=[0.85, 1.0], scale='log'),
              sherpa.Continuous(name="regularization", range=[0.0001, 1], scale='log'),
              sherpa.Continuous(name='dropout', range=[0.0, 0.5]),
              sherpa.Ordinal(name='batch_size', range=[16, 32, 64,128,256,512]),
              sherpa.Choice(name='activation', range=['relu', 'elu', 'prelu', 'tanh'])]

alg =bayesian_optimization.GPyOpt()

study = sherpa.Study(parameters=parameters,
                     algorithm=alg,dashboard_port=9998,disable_dashboard=False,
                     lower_is_better=False)

for trial in study:
    cnn_MFCC_LSTM(study, trial, X_train, X_val, y_train, y_val, regularization =trial.parameters['regularization'],
              drop_likely = trial.parameters['dropout'],
              learning_rate = trial.parameters['lr'],
              beta1 = trial.parameters['beta1'], beta2 = 0.999,
             num_iterations = 150, size_batch=trial.parameters["batch_size"])
    study.finalize(trial)


In [ ]:
def cnn_MFCC_LSTM(study, trial, X_train, X_test, y_train, y_test, regularization = 0.1,drop_likely = 0.1,
              learning_rate = 0.001, beta1 = 0.9, beta2 = 0.999,
             num_iterations = 40, size_batch=X_train.shape[0],activation = 'tanh'):
    # Set the target class number
    
    # VGG 19
    
    target_class = y_train.shape[1]
    # Model 
    model = Sequential()
    
    
    model.add(Conv2D(64, kernel_size = (3,3), activation=activation, input_shape = (50,13,1)))
    model.add(Conv2D(64, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
    model.add(keras.layers.BatchNormalization())
    
    model.add(Conv2D(128, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(128, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
    model.add(keras.layers.BatchNormalization())
    
    model.add(Conv2D(256, kernel_size = (3,3), activation=activation))
    model.add(Conv2D(256, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(256, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(256, kernel_size = (3,3), activation=activation))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
    model.add(keras.layers.BatchNormalization())
    
    
    model.add(Conv2D(512, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(512, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(512, kernel_size = (3,3), activation=activation, padding='same'))
    model.add(Conv2D(512, kernel_size = (3,3), activation=activation))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))
    model.add(keras.layers.BatchNormalization())
    
    
    
    model.add(Flatten())

    
    
    model.add(Dense(4096, activation=activation))
    model.add(Dense(4096, activation=activation))

    model.add(Dense(target_class, activation='softmax'))

    
    
    es = keras.callbacks.EarlyStopping(monitor='val_acc', mode='max',  patience=10, min_delta=0)
    
    opt = keras.optimizers.Adam(lr=learning_rate, beta_1=beta1, beta_2=beta2, epsilon=10e-8)
       
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt,metrics=[keras.metrics.categorical_accuracy])
    
    nn_history = model.fit(X_train, y_train, batch_size=size_batch, epochs=num_iterations, 
                         validation_data=(X_test, y_test),
                          callbacks=[es, study.keras_callback(trial, objective_name='val_categorical_accuracy')])


    return model


In [ ]:
def cnn_inception(X_train, X_test, y_train, y_test):
    #inception architecture
    
    target_class = y_train.shape[1]
    # Model 
    model = Sequential()
    model.add(keras.layers. conv2d_bn(32, 3,3, stridea =(2,2)))
    

In [ ]:
cnn_inception(X_cache, X_cache, Y_cache, Y_cache)

In [ ]:
Y_train = np.argmax(y_train, axis = 1)
Y_val = np.argmax(y_val, axis = 1)

In [ ]:
model = sklearn.svm.NuSVC(gamma = 'auto' )
model.fit(X_train, Y_train)
predictions = model.predict(X_val)
print(classification_report(Y_val, predictions))


In [ ]:
model = sklearn.svm.SVC(gamma = 'scale')
model.fit(X_train, Y_train)
predictions = model.predict(X_val)
print(classification_report(Y_val, predictions))

In [ ]:
model = sklearn.linear_model.SGDClassifier()
model.fit(X_train, Y_train)
predictions = model.predict(X_val)
print(classification_report(Y_val, predictions))

In [ ]:
regularization = 0.1
drop_likely = 0.1
learning_rate = 0.001
beta1 = 0.9 
beta2 = 0.999,
num_iterations = 40
size_batch=256,
activation = 'relu'
target_class = y_train.shape[1]
# Model ALEX NEt
model = Sequential()

model.add(Conv1D(96, 22, padding='valid',strides=2,input_shape=(X_train.shape[1], 1))) 
print(model.input_shape)
print(model.output_shape)
model.add(MaxPooling1D(pool_size = (3), strides=(2)))
print(model.output_shape)
model.add(Conv1D(256, 5, padding='same')) 

print(model.output_shape)
model.add(MaxPooling1D(pool_size = (3), strides=(2)))
print(model.output_shape)

model.add(Conv1D(384, 3, padding='same')) 
print(model.output_shape)

model.add(Conv1D(384, 3, padding='same')) 
print(model.output_shape)

model.add(Conv1D(256, 3, padding='same')) 
print(model.output_shape)

model.add(MaxPooling1D(pool_size = (3), strides=(2)))
print(model.output_shape)
model.add(Flatten())

model.add(Dense(4096, input_dim=9216,  activation=activation,
               kernel_regularizer=regularizers.l2(regularization)))
model.add(keras.layers.BatchNormalization())

model.add(Dense(4096, activation=activation,
               kernel_regularizer=regularizers.l2(regularization)))
model.add(Dropout(drop_likely))

model.add(Dense(2048, activation=activation,
               kernel_regularizer=regularizers.l2(regularization)))
model.add(keras.layers.BatchNormalization())
model.add(Dropout(drop_likely))

model.add(Dense(1024, activation=activation,
               kernel_regularizer=regularizers.l2(regularization)))

model.add(Dense(512, activation=activation,
               kernel_regularizer=regularizers.l2(regularization)))

model.add(Dense(target_class, activation='softmax'))

es = keras.callbacks.EarlyStopping(monitor='val_acc', mode='max',  patience=10, min_delta=0)

opt = keras.optimizers.Adam(lr=learning_rate, beta_1=beta1, beta_2=beta2, epsilon=10e-8)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
nn_history = model.fit(X_train, y_train, batch_size=size_batch, epochs=num_iterations, 
                     validation_data=(X_val, y_val),callbacks=[es])
model, nn_history

In [ ]:
def complex_cnn(X_train, X_test, y_train, y_test):
    # Set the target class number
    target_class = y_train.shape[1]
    # Model 
    model = Sequential()
    model.add(keras.layers.BatchNormalization())
    model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1], 1))) #1
    model.add(Activation('tanh'))
    model.add(Conv1D(256, 8, padding='same')) #2
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.25))
    model.add(Conv1D(128, 8, padding='same')) #3
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh')) 
    model.add(Conv1D(128, 8, padding='same')) #4
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Conv1D(128, 8, padding='same')) #5
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Conv1D(128, 8, padding='same')) #6
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Dropout(0.25))
    model.add(Conv1D(64, 8, padding='same')) #7
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Conv1D(64, 8, padding='same')) #8
    model.add(keras.layers.BatchNormalization())
    model.add(Activation('tanh'))
    model.add(Flatten())
    model.add(Dense(target_class)) #9
    model.add(Activation('softmax'))
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=10e-8, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt,metrics=[keras.metrics.categorical_accuracy])
    cnnhistory = model.fit(X_train, y_train, batch_size=128, epochs=40, 
                         validation_data=(X_test, y_test))
    a = model.predict(X_train)
    predictions = np.zeros_like(a)
    predictions[np.arange(len(a)), a.argmax(1)] = 1
    print(predictions)
    print(y_train)
    print(classification_report(y_train,predictions))
    a = model.predict(X_test)
    predictions = np.zeros_like(a)
    predictions[np.arange(len(a)), a.argmax(1)] = 1
    print(classification_report(y_test,predictions))
    return model, cnnhistory